In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import re
import csv
import cmd
import traceback
import ast
import os
# os.chdir('/content/drive/Shareddrives/USC_DSCI551-  Foundation of Data Management/Project') # where the files for this project are

In [ ]:
# Function to copy selected columns from source CSV to a new CSV
def copyColumns(sourceCsvFile, destinationCsvFile, columnsToCopy):
    try:
        with open(sourceCsvFile, 'r', newline='', encoding='utf-8') as source_file:
            reader = csv.DictReader(source_file)
            source_headers = reader.fieldnames

            # Check if all columns to copy exist in the source CSV
            for column in columnsToCopy:
                if column not in source_headers:
                    print(f"Column '{column}' not found in the source CSV.")
                    return

            with open(destinationCsvFile, 'w', newline='', encoding='utf-8') as dest_file:
                writer = csv.DictWriter(dest_file, fieldnames=columnsToCopy)
                writer.writeheader()

                for row in reader:
                    # Extract the desired columns
                    selected_data = {col: str(row[col]) for col in columnsToCopy}
                    writer.writerow(selected_data)


        print("Selected columns copied to", destinationCsvFile)
        convert_csv_types(destinationCsvFile)

    except FileNotFoundError:
        print("Source CSV file not found.")
    except Exception as e:
        print("An error occurred:", str(e))

def convert_csv_types(fileName):
    try:
        with open(fileName, mode='r', newline='', encoding='utf-8') as input_csv_file:
            reader = csv.reader(input_csv_file)

            updatedData = []
            header = next(reader)  # Read the header row
            data_types = []  # List to store inferred data types for each column

            for column in header:
                if column.replace(".", "", 1).isdigit():
                    data_types.append(float)
                else:
                    data_types.append(str)

            for row in reader:
                converted_row = []
                for value, data_type in zip(row, data_types):
                    if data_type == float:
                        try:
                            converted_item = float(value)
                        except ValueError as e:
                            print(f"Value Error = {e}")
                            converted_item = None  # Keep it as a string for non-numeric values
                    else:
                        converted_item = value  # Keep it as a string
                    converted_row.append(converted_item)
                updatedData.append(converted_row)

        with open(fileName, mode='w', newline='', encoding='utf-8') as output_csv_file:
            writer = csv.writer(output_csv_file)
            writer.writerow(header)  # Write the header row
            writer.writerows(updatedData)

        # print(f"Data types converted and saved to {fileName}")

    except FileNotFoundError:
        print(f"File '{fileName}' not found.")
    except Exception as e:
        print("An error occurred:", str(e))


# loading all the tables
def createIndivisualTables():
    # Essentials Table
    copyColumns("IMDBTop250Movies.csv", "essentials.csv",["rank", "name", "year"])
    copyColumns("IMDBTop250Movies.csv", "viewerDeets.csv",["rank", "rating", "genre", "certificate", "run_time", "tagline"])
    copyColumns("IMDBTop250Movies.csv", "mediaDeets.csv",["rank", "budget", "tagline", "box_office"])
    copyColumns("IMDBTop250Movies.csv", "makers.csv",["rank", "casts", "directors", "writers"])


In [ ]:
# createIndivisualTables()

In [ ]:
class DatabaseFunctions:

        #actual db model functions


        def LoadCsvFile(self, fileName, chunkSize = 10):
            convert_csv_types(fileName)
            table = []
            with open(fileName, mode='r', newline='',  encoding='utf-8') as file:
                reader = csv.reader(file)
                header = next(reader, None)
                if header:
                    for row in reader:
                        data = {header[i]: value for i, value in enumerate(row)}
                        converted_dict = {}

                        for key, value in data.items():
                            try:
                                # Try to convert to int
                                converted_value = int(value)
                            except ValueError:
                                try:
                                    # If not an int, try to convert to float
                                    converted_value = float(value)
                                except ValueError:
                                    # If it can't be converted to a number, keep it as a string
                                    converted_value = value

                            converted_dict[key] = converted_value
                        table.append(converted_dict)

                        if len(table) >= chunkSize:
                            yield table # yeild the chunk of data
                            table = [] # reset table for the next chunk
                else:
                    table = list(reader)
            yield table
            file.close()

        #CREATE TABLE
        def createTable(self, table_name, columns):
            fileName = str(table_name) + ".csv"
            try:
                with open(fileName, mode='w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow(columns)
                    print(f"Table \'{table_name}\' created.")
            except FileNotFoundError:
                print(f"File '{fileName}' not found")
            except Exception as e:
                print("An error occurred:", str(e))

        #INSERT INTO
        def insertInto(self, table_name, columns, values):
            fileName = str(table_name) + ".csv"
            data = [dict(zip(columns, values))]
            try:
                with open(fileName, "a", newline="", encoding='utf-8') as f:
                    for row in data:
                        row_string = ",".join([row[column] for column in columns])
                        f.write(row_string + "\n")

                    convert_csv_types(fileName)
                    print(f"Data inserted into {table_name}")

            except FileNotFoundError:
                print(f"Table '{table_name}' not found. Creating the table...")
                # Create the table with the provided columns
                self.createTable(table_name, columns)
                # Retry the insertion
                self.insertInto(table_name, columns, values)

            except Exception as e:
                print("An error occurred:", str(e))
                traceback.print_exc()

        #GET FROM WHERE
        def get(self, table_name, columns, condition):
            fileName = f"{table_name}.csv"
            try:
                filtered_table = []
                i = 1
                for data in self.LoadCsvFile(fileName, 25):
                    if data == []:
                        break
                    print(f"Chunk {i} processing")
                    i+=1
                    header = list(data[0].keys())
                    if columns == [None]:
                        columns = header
                    for row in data:
                        if eval(condition, row):
                            filtered_table.append({column: row[column] for column in columns})
                    print(*filtered_table, sep='\n')
                return filtered_table

            except FileNotFoundError:
                print(f"Table '{table_name}' not found.")
            except Exception as e:
                print("An error occurred:", str(e))
                traceback.print_exc()

        # show table
        def displayTable(self, table_name):
            fileName = str(table_name) + ".csv"
            table = []
            with open(fileName, mode='r', newline='',  encoding='utf-8') as file:
                reader = csv.reader(file)
                header = next(reader, None)
                if header:
                    i = 0
                    for row in reader:
                        dataRow = {header[i]: value for i, value in enumerate(row)}
                        table.append(dataRow)
                        i+=1
                        if i >=10:
                            break
            print(*table, sep="\n")

        # JOIN
        def joinTable(self, table1_name, table2_name, commonColumns):
            fileName1 = str(table1_name) + ".csv"
            fileName2 = str(table2_name) + ".csv"

            try:
                joinedData = []
                i = 1
                j = 1
                # Load table1 data in chunks
                for chunk1 in self.LoadCsvFile(fileName1, 25):
                    print(f"Chunk {i} of {table1_name} processing")
                    i+=1
                    # Load table2 data in chunks
                    for chunk2 in self.LoadCsvFile(fileName2, 25):
                        print(f"Chunk {j} of {table2_name} processing")
                        j+=1
                        # Perform join operation for each chunk pair
                        chunkJoinedData = []
                        # Join rows from table1 with matching rows from table2
                        for row1 in chunk1:
                            match = False
                            for row2 in chunk2:
                                match = True
                                for key in commonColumns:
                                    if row1.get(key) != row2.get(key):
                                        match = False
                                        break
                                if match:
                                    mergedRow = {**{f'{key}_{table1_name}': value for key, value in row1.items()},
                                                **{f'{key}_{table2_name}': row2.get(key) for key in row2}}
                                    chunkJoinedData.append(mergedRow)
                                    break

                        # Add unmatched rows from table1
                        for row1 in chunk1:
                            match = True
                            for row2 in chunk2:
                                for key in commonColumns:
                                    if row1.get(key) != row2.get(key):
                                        match = False
                                        break
                            if match:
                                mergedRow = {**{f'{key}_{table1_name}': value for key, value in row1.items()},
                                            **{f'{key}_{table2_name}': None for key in commonColumns}}
                                chunkJoinedData.append(mergedRow)

                        joinedData.extend(chunkJoinedData)
                        print(*joinedData, sep='\n')

            except FileNotFoundError:
                print(f"Table(s) '{table1_name}' or '{table2_name}' not found.")
            except Exception as e:
                print("An error occurred:", str(e))
                import traceback
                traceback.print_exc()

        # AGGREGATE FUNCTION
        def aggregate(self, tablename, column, aggFunc, group_by=[]):
            fileName = tablename + ".csv"
            try:
                i = 1
                result = None
              # Load and process data in chunks
                for chunk in self.LoadCsvFile(fileName, 25):
                    print(f"Chunk {i} is being processed")
                    i+=1
                    if chunk != []:
                        if group_by != []:
                            grouped_data = {}
                            for row in chunk:
                                key = tuple(row[col] for col in group_by)
                                if key not in grouped_data:
                                    grouped_data[key] = []
                                grouped_data[key].append(row[column])

                            if aggFunc == "SUM":
                                result_chunk = {key: sum(values) for key, values in grouped_data.items()}
                            elif aggFunc == "AVG":
                                result_chunk = {key: sum(values) / len(values) for key, values in grouped_data.items()}
                            elif aggFunc == "COUNT":
                                result_chunk = {key: len(values) for key, values in grouped_data.items()}
                            elif aggFunc == "MAX":
                                result_chunk = {key: max(values) for key, values in grouped_data.items()}
                            elif aggFunc == "MIN":
                                result_chunk = {key: min(values) for key, values in grouped_data.items()}
                            else:
                                print(f"Unsupported aggregate function: {aggFunc}")
                                return None

                            # Merge results from the current chunk with the overall result
                            if result is None:
                                result = result_chunk
                            else:
                                for key, value in result_chunk.items():
                                    if key in result:
                                        result[key] += value
                                    else:
                                        result[key] = value

                        else:
                            actual_list = [row[column] for row in chunk]
                            if aggFunc == "SUM":
                                result = result + sum(actual_list) if result else sum(actual_list)
                            elif aggFunc == "AVG":
                                if actual_list:
                                    if result is None:
                                        result = {"sum": sum(actual_list), "count": len(actual_list)}
                                    else:
                                        result["sum"] += sum(actual_list)
                                        result["count"] += len(actual_list)
                                else:
                                    # Handle the case where the list is empty
                                    result = result if result else None
                            elif aggFunc == "COUNT":
                                if actual_list:
                                    if result is None:
                                        result = len(actual_list)
                                    else:
                                        result += len(actual_list)
                                else:
                                    # Handle the case where the list is empty
                                    result = result if result is not None else 0
                            elif aggFunc == "MAX":
                                result = max(result, max(actual_list)) if result else max(actual_list)
                            elif aggFunc == "MIN":
                                result = min(result, min(actual_list)) if result else min(actual_list)
                            else:
                                print(f"Unsupported aggregate function: {aggFunc}")
                                return None
                        print(result)

                if aggFunc == "AVG":
                    # Calculate the final AVG after processing all chunks
                    if result and "sum" in result and "count" in result and result["count"] > 0:
                        result = result["sum"] / result["count"]
                    else:
                        result = None
                return result


            except FileNotFoundError:
                print(f"Table '{tablename}' not found.")
            except Exception as e:
                print("An error occurred:", str(e))
                import traceback
                traceback.print_exc()

        #UPDATE FUNCTION
        def update(self, tableName, columns, values, condition):
            fileName = f"{tableName}.csv"

            try:
                updated_rows = []
                i = 1
                for chunk in self.LoadCsvFile(fileName, 25):
                    print(f"Chunk {i} is being processed")
                    i += 1

                    if chunk != []:
                        # Apply the update to the relevant rows in the chunk
                        for row in chunk:
                            # Convert the condition string to a Python expression
                            condition_expr = ast.parse(condition, mode='eval')

                            # Evaluate the expression using eval() in the context of the row
                            condition_result = eval(compile(condition_expr, '<string>', 'eval'), {}, row)

                            # Check if the condition evaluates to True
                            if condition_result:
                                for col, val in zip(columns, values):
                                    row[col] = val

                                updated_rows.append(row)

                        # Add non-updated rows to the updated_rows list
                        for row in chunk:
                            condition_expr = ast.parse(condition, mode='eval')
                            condition_result = eval(compile(condition_expr, '<string>', 'eval'), {}, row)

                            if not condition_result:
                                updated_rows.append(row)

                # Write the updated data back to the file
                with open(fileName, mode='w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)

                    # Write the header
                    writer.writerow(list(updated_rows[0].keys()))

                    # Write the updated rows
                    for updated_row in updated_rows:
                        writer.writerow(list(updated_row.values()))

            except FileNotFoundError:
                print(f"Table '{tableName}' not found.")
            except Exception as e:
                print("An error occurred:", str(e))
                traceback.print_exc()

            print(f"Table {tableName} updated")

        def deleteRow(self, tableName, condition):
            fileName = f"{tableName}.csv"

            try:
                deleted_rows = []
                i = 1
                for chunk in self.LoadCsvFile(fileName, 25):
                    print(f"Chunk {i} is being processed")
                    i += 1

                    if chunk != []:
                        # Apply the deletion to the relevant rows in the chunk
                        for row in chunk:
                            # Convert the condition string to a Python expression
                            condition_expr = ast.parse(condition, mode='eval')

                            # Evaluate the expression using eval() in the context of the row
                            condition_result = eval(compile(condition_expr, '<string>', 'eval'), {}, row)

                            # Check if the condition evaluates to True
                            if condition_result:
                                deleted_rows.append(row)

                # Write the remaining rows back to the file
                with open(fileName, mode='w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)

                    # Write the header
                    writer.writerow(list(chunk[0].keys()))

                    # Write the remaining rows
                    for row in chunk:
                        condition_expr = ast.parse(condition, mode='eval')
                        condition_result = eval(compile(condition_expr, '<string>', 'eval'), {}, row)

                        if not condition_result:
                            writer.writerow(list(row.values()))

                print(f"Deleted {len(deleted_rows)} row(s) from {tableName}")

            except FileNotFoundError:
                print(f"Table '{tableName}' not found.")
            except Exception as e:
                print("An error occurred:", str(e))
                traceback.print_exc()


In [ ]:
class QueryLanguageProcessing:
    def __init__(self):
        self.inputQuery = ""
        self.variables = []

    def getCondition(self, strCondition):
        # Define mapping of words to symbols
        word_to_symbol = {
            'is greater than': '>',
            'is less than': '<',
            'is at least': '>=',
            'is at most': '<=',
            'is equal to': '==',
            'and': 'and',
            'or': 'or'
        }

        # Replace words with corresponding symbols
        for word, symbol in word_to_symbol.items():
            strCondition = strCondition.replace(word, symbol)

        # Replace 'x' and 'y' with their respective variable names
        strCondition = re.sub(r'\bx\b', 'x', strCondition)
        strCondition = re.sub(r'\by\b', 'y', strCondition)

        return strCondition

    def processStatement(self, statement, chunkSize = None):

        # regular expressions to match different parts of the statement
        getPattern = r"GET (.+) FROM TABLE (\w+) (?:WHERE (.+))?" #DONE
        combineTablesPattern = r"COMBINE TABLES (.+) ON (.+)" #DONE
        findAggrigateFuncPattern = r"FIND (\w+) OF COLUMN (.+) FROM TABLE (\w+)" # DONE
        AddValuesIntoPattern = r"ADD VALUES (.+) INTO TABLE (\w+) IN COLUMN/S (.+)" #DONE
        CreateTablePattern = r"CREATE A TABLE (.+) WITH THE FOLLOWING COLUMNS: (.+)" #DONE
        updatePattern = r"UPDATE TABLE (.+) WITH VALUES (.+) IN COLUMN (.+) WHERE (.+)" # DONE
        deletePattern = r"DELETE ROW/S FROM TABLE (.+) WHERE (.+)" # DONE
        showTable = r"SHOW TABLE (.+)" #DONE

        dbFuncObj = DatabaseFunctions()

        #get colums statement
        if re.match(getPattern, statement):
            match = re.match(getPattern, statement)
            columnNames = match.group(1).split(',')
            table = match.group(2)
            if match.group(3) is not None:
                whereConditionStr = match.group(3)
            else:
                whereConditionStr = "True"
            whereCondition = self.getCondition(whereConditionStr)
            print(f"Table Name = {table}, column name = {columnNames}, condition = {whereCondition}")
            # Process the SELECT statement here
            if columnNames == ['ALL']:
                result = dbFuncObj.get(table_name=table, columns=[None], condition=whereCondition)
            else:
                result = dbFuncObj.get(table_name=table, columns=columnNames, condition=whereCondition)

            print(*result, sep="\n")
            # print(result)

        #create table
        elif re.match(CreateTablePattern, statement):
            match = re.match(CreateTablePattern, statement)
            table = match.group(1)
            columns = match.group(2).split(",")
            # Process the CREATE TABLE statement here
            dbFuncObj.createTable(table,columns)

        #combineTable
        elif re.match(combineTablesPattern, statement):
            match = re.match(combineTablesPattern, statement)
            tableNames = match.group(1).split(',')
            columnNames = match.group(2).split(',')
            result = dbFuncObj.joinTable(tableNames[0], tableNames[1], columnNames)
            print(result)
            # if result != []:
              # print(*result, sep="\n")

        #insert value
        elif re.match(AddValuesIntoPattern, statement):
            match = re.match(AddValuesIntoPattern, statement)
            values = match.group(1).split(",")
            tableName = match.group(2)
            columnNames = match.group(3).split(",")
            # Process the CREATE TABLE statement here
            print(f"Table name = {tableName} columns = {columnNames} values = {values}")
            dbFuncObj.insertInto(tableName, columnNames, values)

        #aggrigate function
        elif re.match(findAggrigateFuncPattern, statement):
          match = re.match(findAggrigateFuncPattern, statement)
          aggrigateFunc = match.group(1)
          column = match.group(2)
          tableName = match.group(3)
          #process aggrigate statement here
          print(f"Aggrigate func = {aggrigateFunc} column = {column} table name = {tableName}")
          convert_csv_types(tableName + ".csv")
          result = dbFuncObj.aggregate(tableName, column, aggrigateFunc)
          print(result)

        # update function
        elif re.match(updatePattern, statement):
            match = re.match(updatePattern, statement)
            tableName = match.group(1)
            values = match.group(2).split(",")
            columnNames =  match.group(3).split(",")
            whereConditionStr = match.group(4)
            whereCondition = self.getCondition(whereConditionStr)
            print(f"Table Name = {tableName}, column name = {columnNames}, values = {values}, condition = {whereCondition}")
            dbFuncObj.update(tableName, columnNames, values, whereCondition)

        # delete function
        elif re.match(deletePattern, statement):
            match = re.match(deletePattern, statement)
            tableName = match.group(1)
            whereConditionStr = match.group(2)
            whereCondition = self.getCondition(whereConditionStr)
            print(f"Table Name = {tableName}, condition = {whereCondition}")
            dbFuncObj.deleteRow(tableName, whereCondition)

        #display table
        elif re.match(showTable, statement):
            match = re.match(showTable, statement)
            tableName = match.group(1)
            # Process the CREATE TABLE statement here
            print(f"Table name = {tableName}")
            dbFuncObj.displayTable(tableName)



In [ ]:
# Create the CLI Interface
class MyDbCLI(cmd.Cmd):
        intro = "Welcome to MyDB! Type 'help' to list commands or 'EXIT' to quit."
        prompt = "MyDB> "

        def __init__(self) :
            super().__init__()
            self.dbfunc = DatabaseFunctions()
            self.queryObj = QueryLanguageProcessing()

        def default(self, line):
          self.queryObj.processStatement(line)

        def do_GET(self, line):
          """
          It captures the columns to retrieve, the table name, and an optional WHERE condition for filtering the results.
          SYNTAX:  GET <columns> FROM TABLE <table> WHERE <condition>
                   GET ALL FROM TABLE <table> WHERE <condition>
          EXAMPLE: GET column1,column2 FROM TABLE myTable
                   GET ALL FROM TABLE employees WHERE department is equal to 'HR'

          WHERE CONDITION LIST:
                  is greater than           | >
                  is less than              | <
                  is at least               | >=
                  is at most                | <=
                  is equal to               | ==
                  and                       | and
                  or                        | or
          """
          self.queryObj.processStatement("GET " + str(line))

        def do_COMBINE(self, line):
          """
          This pattern is used to identify a command that combines two or more tables based on a specified column.
          SYNTAX:  COMBINE TABLES <tables> ON <column_name>
          EXAMPLE: COMBINE TABLES table1,table2 ON column
          """
          self.queryObj.processStatement("COMBINE " + str(line))

        def do_FIND(self, line):
          """
          This pattern is designed to identify a query that calculates an aggregate function (e.g., SUM, AVG, COUNT, MAX, MIN) on a specific column within a table.
          SYNTAX:  FIND <aggregate_function> OF COLUMN <column> FROM TABLE <table>
          EXAMPLE: FIND SUM OF COLUMN sales FROM TABLE sales_data
          """
          self.queryObj.processStatement("FIND " + str(line))

        def do_ADD(self, line):
          """
          This pattern captures a command for adding values into specified columns of a table.
          SYNTAX:  ADD VALUES <values> INTO TABLE <table> IN COLUMN/S <columns>
          EXAMPLE: ADD VALUES 'John',30 INTO TABLE employees IN COLUMNS 'name','age'
          """
          self.queryObj.processStatement("ADD " + str(line))


        def do_UPDATE(self, line):
          """
          This pattern is designed to update the values of columns in the given table according to the condition given
          SYNTAX:  UPDATE TABLE <table> WITH VALUES <value/s> IN COLUMN <column/s> WHERE <condition>
          EXAMPLE: UPDATE TABLE testTable WITH VALUES 99,hello!! IN COLUMN a,b WHERE c is equal to 5.3

          WHERE CONDITION LIST:
                  is greater than           | >
                  is less than              | <
                  is at least               | >=
                  is at most                | <=
                  is equal to               | ==
                  and                       | and
                  or                        | or
          """
          self.queryObj.processStatement("UPDATE " + str(line))

        def do_DELETE(self, line):
          """
          This pattern is designed to delete the rows in the given table according to the condition given
          SYNTAX:  DELETE ROW/S FROM TABLE <table> WHERE <condition>
          EXAMPLE: DELETE ROW/S FROM TABLE testTable WHERE c is equal to 5.3

          WHERE CONDITION LIST:
                  is greater than           | >
                  is less than              | <
                  is at least               | >=
                  is at most                | <=
                  is equal to               | ==
                  and                       | and
                  or                        | or
          """
          self.queryObj.processStatement("DELETE " + str(line))

        def do_CREATE(self, line):
          """
          This pattern is used for creating a new table with a list of specified columns.
          SYNTAX:  CREATE A TABLE <table> WITH THE FOLLOWING COLUMNS: <columns>
          EXAMPLE: CREATE A TABLE products WITH THE FOLLOWING COLUMNS: product_id,product_name,price
          """
          self.queryObj.processStatement("CREATE " + str(line))

        def do_SHOW(self, line):
          """
          This pattern captures a command to display first 10 rows of a specific table
          SYNTAX:  SHOW TABLE <table>
          EXAMPLE: SHOW TABLE customers
          """
          print(line)
          self.queryObj.processStatement("SHOW " + str(line))

        def do_help(self, args):
          """
          Display help for available commands.
          SYNTAX: help [<command_name>]
          EXAMPLE: help CREATE
          """
          if args:
              try:
                  command = getattr(self, f"do_{args}")
                  print(command.__doc__)
              except AttributeError:
                  print(f"Command '{args}' not found.")
          else:
              super().do_help(args)

        def do_exit(self, args):
          """
          Exit the MyDB CLI!
          """
          print("Exiting MyDB. Goodbye!")
          return True

if __name__ == "__main__":
  myCliObj = MyDbCLI()
  myCliObj.cmdloop()
